In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
cd /content/drive/My\ Drive/assign2

In [0]:
import re
import nltk
import pandas as pd
from keras.preprocessing.text import Tokenizer
from nltk.tokenize import word_tokenize
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed,Embedding,Dropout,Conv1D,MaxPool1D
from keras.utils import to_categorical
import numpy as np
from numpy import array
import math
nltk.download('punkt')

In [0]:
data = pd.read_csv('train.txt', delimiter="\t",header=None,names=["meta", "uid", "senti"])
meta=data['meta']
uid=data['uid']
senti=data['senti']
X_train=[]
Y_train=[]
temp=[]
dic={'negative':0,'positive':1,'neutral':2}
for i,j,k in zip(meta,uid,senti):
    if i=='meta':
        if len(temp)!=0:
            X_train.append(temp)
        Y_train.append(dic[k])
        temp=[]
    else:
        try:
          if j!='O':
            i=re.sub(r'[\t\n]+','',i)
            temp.append(i)
        except:
            pass
X_train.append(temp)

In [0]:
new=[]
for i in X_train:
    temp=[]
    for j in i:
        temp.extend(list(j))
    new.append(temp)

In [97]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(new)
x_train = tokenizer.texts_to_sequences(new)
maxx=0
for i in x_train:
    if maxx<len(i):
        maxx=len(i)
        maxi=i
x1_train=[]
maxx=maxx+1
for i in x_train:
  i.extend([0]*(maxx-len(i)))
  x1_train.append(i)

13524

In [0]:
x1_train=array(x1_train)
y1_train=to_categorical(Y_train, num_classes=3)

In [99]:
x1_train,y1_train,maxx

(array([[ 1, 13,  2, ...,  0,  0,  0],
        [11,  1, 13, ...,  0,  0,  0],
        [ 8,  1,  7, ...,  0,  0,  0],
        ...,
        [ 1, 11, 20, ...,  0,  0,  0],
        [ 9, 14,  8, ...,  0,  0,  0],
        [10, 14,  8, ...,  0,  0,  0]]), array([[1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        ...,
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]], dtype=float32), 198)

In [100]:
vocab_size=len(tokenizer.word_index)+1
vocab_size

94

In [101]:
embedding_weights=[]
embedding_weights.append(np.zeros(vocab_size))
embedding_weights

for char, i in tokenizer.word_index.items():
    onehot=np.zeros(vocab_size)
    onehot[i-1] =1
    embedding_weights.append(onehot)
embedding_weights=np.array(embedding_weights)
embedding_weights

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [0]:
input_size=maxx
embedding_size=94
conv_layers=[[256,7,3],
             [256,3,-1],
             [256,3,-1],
             [256,3,-1],
             [256,3,3]]
fully_connected_layers=[1024,1024]
num_of_classes=3
dropout_p=0.3
optimizer='adam'
loss='categorical_crossentropy'

In [0]:
from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model

In [0]:
embedding_layer=Embedding(vocab_size,embedding_size,input_length=input_size,weights=[embedding_weights])

In [0]:
inputs=Input(shape=(input_size,),name='input',dtype='float32')

x=embedding_layer(inputs)

for filter_num,filter_size,pooling_size in conv_layers:
    x= Conv1D(filter_num,filter_size)(x)
    x=Activation('relu')(x)
    if pooling_size!=-1:
        x=MaxPooling1D(pool_size=pooling_size)(x)
x=Flatten()(x)
for dense_size in fully_connected_layers:
    x=Dense(dense_size,activation='relu')(x)
    x=Dropout(dropout_p)(x)
predictions = Dense(num_of_classes,activation='softmax')(x)
model=Model(inputs=inputs,outputs=predictions)
model.compile(optimizer=optimizer,loss=loss,metrics=['accuracy'])

In [0]:
# model = Sequential()
# model.add(Embedding(94, 128, input_length=197))
# model.add(Conv1D(128,kernel_size=3))
# model.add(MaxPool1D(pool_size=3))
# model.add(LSTM(128,return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(128))
# model.add(Dropout(0.2))
# model.add(Dense(3, activation='softmax'))
# model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

In [135]:
model.fit(x1, y1, batch_size=32, epochs=80,verbose=1)

Epoch 1/80
13524/13524 [==============================] - 5s 376us/step - loss: 0.0427 - acc: 0.9881
Epoch 2/80
13524/13524 [==============================] - 5s 379us/step - loss: 0.0472 - acc: 0.9854
Epoch 3/80
13524/13524 [==============================] - 5s 369us/step - loss: 0.0484 - acc: 0.9871
Epoch 4/80
13524/13524 [==============================] - 5s 364us/step - loss: 0.0520 - acc: 0.9864
Epoch 5/80
13524/13524 [==============================] - 5s 360us/step - loss: 0.0469 - acc: 0.9860
Epoch 6/80
13524/13524 [==============================] - 5s 359us/step - loss: 0.0301 - acc: 0.9913
Epoch 7/80
13524/13524 [==============================] - 5s 364us/step - loss: 0.0369 - acc: 0.9891
Epoch 8/80
13524/13524 [==============================] - 5s 362us/step - loss: 0.0422 - acc: 0.9868
Epoch 9/80
13524/13524 [==============================] - 5s 363us/step - loss: 0.0538 - acc: 0.9866
Epoch 10/80
13524/13524 [==============================] - 5s 362us/step - loss: 0.0585 - a

In [0]:
model.save("model_testgood.h5")

In [0]:
from keras.models import load_model
model1=load_model("model_testgood.h5")

**EXTRACTING TEST DATA**

In [0]:
data1 = pd.read_csv('test.txt', delimiter="\t",header=None,names=["meta", "uid", "senti"])
meta1=data1['meta']
uid1=data1['uid']
senti1=data1['senti']
X1_test=[]
Y1_test=[]
temp1=[]
dic1={'negative':0,'positive':1,'neutral':2}
for i,j,k in zip(meta1,uid1,senti1):
    if i=='meta':
        if len(temp1)!=0:
            X1_test.append(temp1)
        Y1_test.append(dic1[k])
        temp1=[]
    else:
        try:
          if j!='O':
            i=re.sub(r'[\t\n]+','',i)
            temp1.append(i)
        except:
            pass
X1_test.append(temp1)

In [0]:
new1=[]
for i in X1_test:
    temp1=[]
    for j in i:
        temp1.extend(list(j))
    new1.append(temp1)

In [0]:
tokenizer.fit_on_texts(new1)
x2_test = tokenizer.texts_to_sequences(new1)

x3_test=[]
for i in x2_test:
  i.extend([0]*(maxx-len(i)))
  x3_test.append(i)


In [132]:
x3_test=array(x3_test)
y3_test=to_categorical(Y1_test, num_classes=3)
x3_test[0]

array([ 7,  5, 14,  1,  1, 15, 18,  6,  9, 22,  3,  8,  8,  2,  6,  9,  8,
       12,  6, 23,  3, 12,  6,  6, 10,  8, 19,  6,  6, 13,  6,  9, 11,  1,
       13, 13,  2,  3,  1, 10,  6, 12,  1,  9, 19, 16,  1, 17, 14,  9, 19,
        8,  6, 16,  7,  1,  9, 19, 11,  1,  8,  1, 17,  1, 10,  1,  8,  2,
       11,  1, 17, 24,  6, 12,  6,  8, 17,  1, 16,  3, 22,  6,  7,  3, 21,
        2,  5,  4,  5,  4,  3, 15,  1,  8,  5, 24, 11,  3, 13, 17,  6, 12,
        6, 21, 10,  3, 17,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [134]:
from sklearn.metrics import classification_report,accuracy_score
yout=array(Y1_test)
y_predict = model1.predict(x3, verbose=0)
Y_predict=np.argmax(yhat,axis=1)
print(classification_report(Y1_test,Y_predict))  #['negative':0,'positive':1,'neutral':2]

              precision    recall  f1-score   support

           0       0.56      0.37      0.45       533
           1       0.53      0.43      0.47       582
           2       0.46      0.64      0.53       754

    accuracy                           0.50      1869
   macro avg       0.52      0.48      0.49      1869
weighted avg       0.51      0.50      0.49      1869

